In [1]:
!pip install -U bitsandbytes

In [8]:
%load_ext rich

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [3]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

In [4]:
OPT = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b",quantization_config = quantization_config)
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [5]:
inp = "The quick brown fox jumps over the lazy dog"
inp_tokenized = tokenizer(inp, return_tensors="pt")

In [6]:
inp_tokenized["input_ids"].size()

torch.Size([1, 10])

In [9]:
OPT.model


OPTModel(
  (decoder): OPTDecoder(
    (embed_tokens): Embedding(50272, 2048, padding_idx=1)
    (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
    (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0-23): 24 x OPTDecoderLayer(
        (self_attn): OPTAttention(
          (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (out_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
        )
        (activation_fn): ReLU()
        (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear8bitLt(in_features=8192, out_features=2048, bias=True)
        (final_layer_norm): LayerNorm((2048,), eps=1e-05, 

In [11]:
embedded_input = OPT.model.decoder.embed_tokens(inp_tokenized["input_ids"])
embedded_input


tensor([[[-0.0407,  0.0519,  0.0574,  ..., -0.0263, -0.0355, -0.0260],
         [-0.0371,  0.0220, -0.0096,  ...,  0.0265, -0.0166, -0.0030],
         [-0.0455, -0.0236, -0.0121,  ...,  0.0043, -0.0166,  0.0193],
         ...,
         [ 0.0007,  0.0267,  0.0257,  ...,  0.0622,  0.0421,  0.0279],
         [-0.0126,  0.0347, -0.0352,  ..., -0.0393, -0.0396, -0.0102],
         [-0.0115,  0.0319,  0.0274,  ..., -0.0472, -0.0059,  0.0341]]],
       dtype=torch.float16, grad_fn=<EmbeddingBackward0>)

In [12]:
embed_pos_input = OPT.model.decoder.embed_positions(inp_tokenized["attention_mask"])
embed_pos_input


tensor([[[-8.1406e-03, -2.6221e-01,  6.0768e-03,  ...,  1.7273e-02,
          -5.0621e-03, -1.6220e-02],
         [-8.0585e-05,  2.5000e-01, -1.6632e-02,  ..., -1.5419e-02,
          -1.7838e-02,  2.4948e-02],
         [-9.9411e-03, -1.4978e-01,  1.7557e-03,  ...,  3.7117e-03,
          -1.6434e-02, -9.9087e-04],
         ...,
         [ 3.6979e-04, -7.7454e-02,  1.2955e-02,  ...,  3.9330e-03,
          -1.1642e-02,  7.8506e-03],
         [-2.6779e-03, -2.2446e-02, -1.6754e-02,  ..., -1.3142e-03,
          -7.8583e-03,  2.0096e-02],
         [-8.6288e-03,  1.4233e-01, -1.9012e-02,  ..., -1.8463e-02,
          -9.8572e-03,  8.7662e-03]]], dtype=torch.float16,
       grad_fn=<EmbeddingBackward0>)

In [14]:
embed_position_input = embedded_input+embed_pos_input
hidden_states, _= OPT.model.decoder.layers[0].self_attn(embed_position_input)

In [16]:
embed_position_input


tensor([[[-0.0488, -0.2103,  0.0635,  ..., -0.0090, -0.0406, -0.0422],
         [-0.0372,  0.2720, -0.0262,  ...,  0.0111, -0.0345,  0.0220],
         [-0.0555, -0.1735, -0.0104,  ...,  0.0080, -0.0331,  0.0183],
         ...,
         [ 0.0010, -0.0507,  0.0387,  ...,  0.0661,  0.0305,  0.0357],
         [-0.0153,  0.0123, -0.0520,  ..., -0.0406, -0.0474,  0.0099],
         [-0.0202,  0.1743,  0.0084,  ..., -0.0657, -0.0157,  0.0429]]],
       dtype=torch.float16, grad_fn=<AddBackward0>)